# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result, find_tuning_curves, raster_plot, more_tuning_curves, trial_aligned_tuning_curves
from neural_data_analysis.neural_analysis_tools.test_neural_response import compare_firing_rate
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_decoding_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca
import quantities as pq

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# tuning curves

## prep data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"

reduce_y_var_lags = False
planning_data_by_point_exists_ok = True
y_data_exists_ok = False

pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok,
                                 add_behav_data=False,
                                 put_data_into_bins=False)

pn.prepare_seg_aligned_data()
pn.get_concat_data_for_regression(use_raw_spike_data_instead=True,
                                  apply_pca_on_raw_spike_data=False,
                                  use_lagged_raw_spike_data=False) 

## fit + plot curves

In [ ]:
pn.concat_behav_trials.columns

### cur_ff_rel_x

In [ ]:
var_of_interest = 'cur_ff_rel_x'
find_tuning_curves.compute_and_plot_tuning_curves_pooled(pn.concat_neural_trials, pn.concat_behav_trials, var_of_interest,
                                                    n_stimulus_bins=10, variable_type='continuous',
                                                    binning_method='quantile',
                                                    exclude_edge_bins=True)

### cur_ff_distance

In [ ]:
var_of_interest = 'cur_ff_distance'
find_tuning_curves.compute_and_plot_tuning_curves_pooled(pn.concat_neural_trials, pn.concat_behav_trials, var_of_interest,
                                                    n_stimulus_bins=10, variable_type='continuous',
                                                    binning_method='quantile',
                                                    exclude_edge_bins=False)

### cur_ff_angle

In [ ]:
var_of_interest = 'cur_ff_angle'
find_tuning_curves.compute_and_plot_tuning_curves_pooled(pn.concat_neural_trials, pn.concat_behav_trials, var_of_interest,
                                                    n_stimulus_bins=15, variable_type='continuous',
                                                    binning_method='quantile',
                                                    exclude_edge_bins=True)

### nxt_ff_distance

In [ ]:
var_of_interest = 'nxt_ff_distance'
find_tuning_curves.compute_and_plot_tuning_curves_pooled(pn.concat_neural_trials, pn.concat_behav_trials, var_of_interest,
                                                    n_stimulus_bins=7, variable_type='continuous',
                                                    binning_method='quantile',
                                                    exclude_edge_bins=False)

### nxt_ff_rel_x

In [ ]:
var_of_interest = 'nxt_ff_rel_x'
find_tuning_curves.compute_and_plot_tuning_curves_pooled(pn.concat_neural_trials, pn.concat_behav_trials, var_of_interest,
                                                    n_stimulus_bins=10, variable_type='continuous',
                                                    binning_method='quantile',
                                                    exclude_edge_bins=True)

## fit models for tuning curves

In [ ]:
more_tuning_curves.fit_cosine_tuning_curves(pn.concat_neural_trials, pn.concat_behav_trials, 'cur_ff_angle')

# trial-averaged tuning curves

In [ ]:
# # distribute data into new_segments and add rel_spike_time
# pn.get_new_seg_info(cur_or_nxt='nxt', first_or_last='first', time_limit_to_count_sighting=2, start_t_rel_event=0.75, end_t_rel_event=1.25, exists_ok=False)

In [ ]:
time_before_event = 0.9
time_after_event = 0.9
n_bins = 6

new_seg_info2 = pn.new_seg_info.copy()
new_seg_info2['new_seg_start_time'] = new_seg_info2['event_time'] - time_before_event
new_seg_info2['new_seg_end_time'] = new_seg_info2['event_time'] + time_after_event
aligned_spike_trains = raster_plot.prepare_aligned_spike_trains(new_seg_info2, pn.spikes_df)

In [ ]:
duration = time_before_event + time_after_event
last_bin_index = n_bins - 1
event_x_position = time_before_event/duration * last_bin_index
aligned_spike_trains = trial_aligned_tuning_curves.prepare_trial_aligned_tuning_curves_data(aligned_spike_trains)
firing_df, avg_firing_df = trial_aligned_tuning_curves.trial_averaged_time_normalized_firing(aligned_spike_trains, 
                                                                                            n_bins=n_bins, 
                                                                                            clusters_per_fig=3,
                                                                                            rescale_avg_firing_rate=True,
                                                                                            event_x_position=event_x_position)